# ISPA - SearchLight
Script adapted from: https://github.com/SylvainTakerkart/inter_subject_pattern_analysis/tree/master/fmri_data

Original paper: https://doi.org/10.1016/j.neuroimage.2019.116205

This script demonstrates how to use the searchlight implementation
available in nilearn to perform group-level decoding using an
inter-subject pattern analysis (ISPA) scheme.

## Importing packages

In [ ]:
import pandas as pd
import numpy as np
import os
import os.path as op
import re
import glob
import tarfile
import urllib

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.linear_model import LogisticRegression

from nilearn.image import math_img, new_img_like, concat_imgs, resample_to_img, clean_img
from nilearn.decoding import SearchLight
from nilearn.image import math_img, new_img_like, concat_imgs, resample_to_img
from nilearn.maskers import NiftiMasker
from nilearn.masking import apply_mask, compute_epi_mask
from nilearn import plotting

import nibabel as n

from concurrent.futures import ProcessPoolExecutor, as_completed
import random
import csv
import time  # Import the time module


## Opening files to work with

In [ ]:
# Set the path to your GLM directory
data_dir = '/projetos/PRJ1901_AMID/03_PROCS/Postnatal_Affective_Dataset/PROC_DATA/derivatives/GLM'

# getting a full list of available subjects
files = os.listdir(data_dir)
pattern = r'sub-(\d+)_LSA'
subject_numbers = [re.search(pattern, file_name).group(1) for file_name in files if re.search(pattern, file_name)]

# List of subjects to include, empty means include all
subjects_to_include = ['3120','0196']  # Fill with subject identifiers or leave empty to include all

beta_flist = []
y = []
subj_vect = []
runs = []
        
# Create lists from beta map filenames in 'beta_flist', labels in 'y', and subject numbers in 'subj_vect'.
for subject_number in subject_numbers:
    # If subjects_to_include is empty or subject_number is in the list, process the subject
    if not subjects_to_include or subject_number in subjects_to_include:
        subject_directory = os.path.join(data_dir, f'sub-{subject_number}_LSA')

        # List beta map files for the current subject
        beta_map_files = glob.glob(os.path.join(subject_directory, f'sub-{subject_number}_run*_*.nii.gz'))
        beta_map_files.sort()
        beta_flist.extend(beta_map_files)

        # Extract the label from the beta map filename
        for beta_map_file in beta_map_files:
            label = beta_map_file.split('_')[-3]  # Assumes label is the third last part of the filename
            if "InfOther" in label:  # Adjust labels to consider Own vs. Other (irrelevant to valence)
                label = "Other"
            elif "InfOwn" in label:
                label = "Own"
            y.append(label)
            subj_vect.append(subject_number)

        # Extract run info from the beta map filename (to be used in standardization)
        for beta_map_file in beta_map_files:
            run = beta_map_file.split('_')[-4]  # Assumes run is the fourth last part of the filename
            runs.append(run)

## STANDARDIZATIOIN (per run and subject)

# read image data
print("Reading beta maps from all the subjects...")

fmri_nii_dict = {}

for beta_path in beta_flist:
    parts = re.split(r'[/_]', beta_path)
    subj = parts[-6]
    run = 'run-'+parts[-4]
    condition = parts[-3]
    beta = parts[-2]
    
    image = nb.load(beta_path)
    
    # Check if the subject already exists in the dictionary
    if subj in fmri_nii_dict:
        # If the run exists for the subject, append the data to the existing list
        if run in fmri_nii_dict[subj]:
            fmri_nii_dict[subj][run].append({'condition': condition, 'beta': beta, 'image': image})
        # If the run doesn't exist, create a new entry for the run
        else:
            fmri_nii_dict[subj][run] = [{'condition': condition,'beta': beta, 'image': image}]
    # If the subject doesn't exist, create a new entry for the subject and run
    else:
        fmri_nii_dict[subj] = {run: [{'condition': condition,'beta': beta, 'image': image}]}

# Printing the created dictionary
for subj, subj_value in fmri_nii_dict.items():
    for run, run_value in subj_value.items():
        print(f"Subject: {subj}, Run: {run}")
        for item in run_value:
            print(item)
        print('\n')
        
## CONCATENATE BETAS IMAGES ##

concat_imgs_dict = {}
concat_imgs_stand_dict = {}

# Concatenating the images for the same run and subject
for subj, subj_value in fmri_nii_dict.items():
    for run, run_value in subj_value.items():
        images_to_concat = [item['image'] for item in run_value]
        concatenated_image = concat_imgs(images_to_concat)
        print(f"Subject: {subj}, Run: {run}, Concatenated Beta Images Shape: {concatenated_image.shape}")
        
        # Adding concatenated image to the concat_imgs_dict
        if subj in concat_imgs_dict:
            concat_imgs_dict[subj][run] = concatenated_image
        else:
            concat_imgs_dict[subj] = {run: concatenated_image}

        # Standardized beta maps with clean_img()
        stand_image = clean_img(concatenated_image, standardize=True, detrend=False)
        print(f"Subject: {subj}, Run: {run}, Concatenated Standardized Beta Images Shape: {stand_image.shape}")
        
        # Adding standardized image to the concat_imgs_stand_dict
        if subj in concat_imgs_stand_dict:
            concat_imgs_stand_dict[subj][run] = stand_image
        else:
            concat_imgs_stand_dict[subj] = {run: stand_image}
            
# Concatenate all subjects #

all_images_to_concat = []

for subj, subj_value in concat_imgs_stand_dict.items():
    for run, stand_image in subj_value.items():
        all_images_to_concat.append(stand_image)

all_betas_stand = concat_imgs(all_images_to_concat) 

print(f"All Betas Stand Shape: {all_betas_stand.shape}")

# Setting up for leave-one-subject-out cross-validation 

In [ ]:
# set up leave-one-subject-out cross-validation
loso = LeaveOneGroupOut()
unique_subjects = np.unique(subj_vect)
n_splits = loso.get_n_splits(groups=unique_subjects)

In [ ]:
# output directory
output_dir = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut_test"
os.makedirs(output_dir, exist_ok=True)

# Calculate the chance level
chance_level = 1. / len(np.unique(y))

# reading brain mask
brain_mask = nb.load("/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/brain_mask.nii.gz")

# Check whether mask and Betas are in the same space and resample if necessary
beta_img = nb.load(beta_map_files[0])

if not beta_img.shape == brain_mask.shape:
    brain_mask = resample_to_img(brain_mask, beta_img, interpolation='nearest')
    print("Brain mask resampled to match betas images.")
    
whole_brain = 1

if whole_brain == 1:
    print("running whole-brain classification")
else:
    # open ROI mask if not  whole-brain
    roi_mask_nii = nb.load("/projetos/PRJ1901_AMID/03_PROCS/Postnatal_Affective_Dataset/BIDS_Nilearn/2ndLvPilotSPACE_Binar_NAccBrian_OFC_Septal.nii")
    # Check whether ROI and Betas are in the same space and resample if necessary
    if not beta_img.shape == roi_mask_nii.shape:
        roi_mask_nii = resample_to_img(roi_mask_nii, beta_img, interpolation='nearest')
        print("ROI mask resampled to match the beta images.")
        print("running ROI classification")
        
# running searchlight decoding
searchlight_radius = 4
n_jobs = -1
y = np.array(y)
single_split_path_list = []
print("Launching cross-validation...")

for split_ind, (train_inds,test_inds) in enumerate(loso.split(subj_vect,subj_vect,subj_vect)):
    print("...split {:02d} of {:02d}".format(split_ind+1, n_splits))
    single_split = [(train_inds,test_inds)]
    y_train = y[train_inds]
    n_samples = len(y_train)
    class_labels = np.unique(y_train)
    clf = LogisticRegression()
    if whole_brain == 1:
        searchlight = SearchLight(mask_img = brain_mask,
                                  process_mask_img = brain_mask,
                              radius=searchlight_radius,
                              n_jobs=n_jobs,
                              verbose=0,
                              cv=single_split,
                              estimator=clf)
    else:
        searchlight = SearchLight(mask_img = brain_mask,
                              radius=searchlight_radius,
                              n_jobs=n_jobs,
                              verbose=0,
                              cv=single_split,
                              estimator=clf,
                              process_mask_img=roi_mask_nii)
            
    
    searchlight.fit(all_betas_stand, y)
    
    print("...mapping the data (this takes a long time) and fitting the model in each sphere")
    
    single_split_nii = new_img_like(brain_mask,searchlight.scores_ - chance_level)
    single_split_path = op.join(output_dir, 'ispa_searchlight_accuracy_split{:02d}of{:02d}.nii'.format(split_ind+1,n_splits))
    print('Saving score map for cross-validation fold number {:02d}'.format(split_ind+1))
    single_split_nii.to_filename(single_split_path)
    single_split_path_list.append(single_split_path)

# Non-parametric Permutations with nilearn
Originally it was performed with SnPM MATLAB toolbox to analyse the single-fold (for the inter-subject cross-validation of ISPA) accuracy maps, with 1000 permutations and a significance threshold (p<0,05, FWE corrected).
Here, we are trying to do the same, but using nilearn.

Original SnPM batch: https://github.com/SylvainTakerkart/inter_subject_pattern_analysis/blob/master/fmri_data/snpm_batch.m

## Here we have tried to implement the permutatoin scheme suggested by Stelzer et al. 2013, by permuting labels for X searchlights per subject

In [132]:
# output directory
output_dir = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut_test/withinsubj_permut_test"
os.makedirs(output_dir, exist_ok=True)

# Calculate the chance level
chance_level = 1. / len(np.unique(y))

# running searchlight decoding
searchlight_radius = 4
n_jobs = -1
y = np.array(y)
single_split_path_list = []

# Number of permutations within subject
n_permutations = 2

# List to hold the timings
timings = []

# List to hold the mean accuracies per permutation and split
mean_accs = []

# reading brain mask
brain_mask = nb.load("/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/brain_mask.nii.gz")

# Check whether mask and Betas are in the same space and resample if necessary
beta_img = nb.load(beta_map_files[0])

if not beta_img.shape == brain_mask.shape:
    brain_mask = resample_to_img(brain_mask, beta_img, interpolation='nearest')
    print("Brain mask resampled to match betas images.")
    
whole_brain = 1

if whole_brain == 1:
    print("running whole-brain classification")
else:
    # open ROI mask if not  whole-brain
    roi_mask = nb.load("/projetos/PRJ1901_AMID/03_PROCS/Postnatal_Affective_Dataset/BIDS_Nilearn/2ndLvPilotSPACE_Binar_NAccBrian_OFC_Septal.nii")
    # Check whether ROI and Betas are in the same space and resample if necessary
    if not beta_img.shape == roi_mask.shape:
        roi_mask = resample_to_img(roi_mask, beta_img, interpolation='nearest')
        print("ROI mask resampled to match the beta images.")
        print("running ROI classification")

        
# Initialize NiftiMasker with the brain mask, and fit it once
nifti_masker = NiftiMasker(standardize=False,
            mask_img=brain_mask,
            memory_level=0,
            smoothing_fwhm=None)

nifti_masker.fit()        
        
        
print("Launching cross-validation...")
total_start_time = time.time()  # Capture the start time of the entire process

for split_ind, (train_inds,test_inds) in enumerate(loso.split(subj_vect,subj_vect,subj_vect)):
    print("...split {:02d} of {:02d}".format(split_ind+1, n_splits))
    split_start_time = time.time()  # Start time for this split
    
    for permutation in range(n_permutations):
        print(f"...within-subject permutation {permutation+1:02d} of {n_permutations}")
        permutation_start_time = time.time()  # Start time for this permutation
        
        # Append the results to timing list
        #timings.append({f"Start time for split {split_ind+1}': split_start_time})
        
        # Shuffle y labels for this permutation
        np.random.seed(permutation)  # Setting the seed for reproducibility
        y_shuffled = np.random.permutation(y)

        # Set up the classifier and searchlight with the shuffled labels
        clf = LogisticRegression()
        if whole_brain == 1:
            searchlight = SearchLight(mask_img = brain_mask,
                                  process_mask_img = brain_mask,
                              radius=searchlight_radius,
                              n_jobs=n_jobs,
                              verbose=0,
                              cv=single_split,
                              estimator=clf)
        else:
            searchlight = SearchLight(mask_img = brain_mask,
                                  process_mask_img = roi_mask,
                              radius=searchlight_radius,
                              n_jobs=n_jobs,
                              verbose=0,
                              cv=single_split,
                              estimator=clf)
        
        searchlight_start_time = time.time()  # Start time for this searchlight
        # Fit the model with shuffled labels
        searchlight.fit(all_betas_stand, y_shuffled)

        # Compute accuracy map and subtract chance level
        accuracy_map = searchlight.scores_ - chance_level

        # Save the accuracy map for this permutation and split
        single_split_nii = new_img_like(brain_mask, accuracy_map)
        output_filename = f'split_{split_ind+1:02d}_permutation_{permutation+1:03d}_accuracy.nii'
        single_split_path = os.path.join(output_dir, output_filename)
        print(f'Saving permutation score map {permutation+1:03d} for split {split_ind+1:02d}')
        single_split_nii.to_filename(single_split_path)
        
        ####INCLUDE MASK TO GET ONLY BRAIN VALUES!!!! ######
    
        # Apply the mask to your statistical image
        masked_data = apply_mask(single_split_nii, nifti_masker.mask_img_)
    
        # Get the shape of the statistical image
        map_data = single_split_nii.get_fdata()
        stat_shape = map_data.shape

        # Create an empty array of the same shape as the statistical image
        masked_img_data = np.zeros(stat_shape)

        # Fill the masked_img_data with masked_data
        masked_img_data[nifti_masker.mask_img_.get_fdata() > 0] = masked_data
        # create masked nii
        masked_img = nib.Nifti1Image(masked_img_data, affine=single_split_nii.affine)
        #save masked image
        output_filename_mask = f'split_{split_ind+1:02d}_permutation_{permutation+1:03d}_accuracy_MASKED.nii'
        single_split_path_masked = os.path.join(output_dir, output_filename_mask)
        masked_img.to_filename(single_split_path_masked)
        
        # Append the split number, permutation number, and mean accuracy to mean_accs
        mean_accs.append({
            "Split Number": split_ind + 1,
            "Permutation Number": permutation + 1,
            "Mean Accuracy": (np.mean(masked_data))
        })
        
        # Print duration for this searchlight
        searchlight_end_time = time.time()
    
        print(f"Searchlight {permutation+1} completed in {searchlight_end_time - searchlight_start_time:.2f} seconds.")
        # Append the results to timing list
        timings.append({f"Searchlight {permutation+1}: {searchlight_end_time - searchlight_start_time:.2f}"})
        
    # Print duration for this split
    split_end_time = time.time()
    print(f"Split {split_ind+1} completed in {split_end_time - split_start_time:.2f} seconds.")
    timings.append({f"Split {split_ind+1}: {split_end_time - split_start_time:.2f}"})

# Print the total duration
total_end_time = time.time()
print(f"Total processing time: {total_end_time - total_start_time:.2f} seconds.")
timings.append({f"Total processing time: {total_end_time - total_start_time:.2f}"})

new_df = pd.DataFrame(timings)
new_df.to_csv('timings_all.csv', index=False)

# Create a DataFrame from mean_accs
mean_accs_df = pd.DataFrame(mean_accs)

# Save to CSV
mean_accs_csv_path = os.path.join(output_dir, "null_array_100_perm.csv")
mean_accs_df.to_csv(mean_accs_csv_path, index=False)

Brain mask resampled to match betas images.
running whole-brain classification
Launching cross-validation...
...split 01 of 02
...within-subject permutation 01 of 2
Saving permutation score map 001 for split 01
Searchlight 1 completed in 88.34 seconds.
...within-subject permutation 02 of 2
Saving permutation score map 002 for split 01
Searchlight 2 completed in 84.87 seconds.
Split 1 completed in 173.21 seconds.
...split 02 of 02
...within-subject permutation 01 of 2
Saving permutation score map 001 for split 02
Searchlight 1 completed in 84.78 seconds.
...within-subject permutation 02 of 2
Saving permutation score map 002 for split 02
Searchlight 2 completed in 85.51 seconds.
Split 2 completed in 170.29 seconds.
Total processing time: 343.50 seconds.


## Randomly select one accuracy map per subject and average (repeat 10n5 x).

## Pool of 10n5 chance group accuracy maps


In [149]:
# Define input and output directories
input_dir = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut_test/withinsubj_permut_test"
output_dir = os.path.join(input_dir, "permu_mean_group_acc_maps_test")
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Function to extract unique split identifiers based on existing files in directory
def get_unique_identifiers(directory, pattern):
    files = glob.glob(os.path.join(directory, pattern))
    unique_ids = set()
    for file in files:
        match = re.search(r"split_(\d+)_permutation_(\d+)_accuracy\.nii", file)
        if match:
            unique_ids.add(match.group(1))
    return list(unique_ids)

# Use the function to get a list of unique split identifiers
split_identifiers = get_unique_identifiers(input_dir, "split_*_permutation_*_accuracy.nii")

# fix identifier for testing
split_identifiers= ['01','02']

# Preload maps paths for each split identifier
split_maps_paths = {}
for split_identifier in split_identifiers:
    pattern = os.path.join(input_dir, f"split_{split_identifier}_permutation_*_accuracy.nii")
    split_maps_paths[split_identifier] = glob.glob(pattern)

# Load an example NIfTI image to get the shape and affine
example_img_path = split_maps_paths[split_identifiers[0]][0]
example_img = nib.load(example_img_path)

# load brain mask for computation of mean values considering only "brain voxels"
brain_mask = nb.load("/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/brain_mask.nii.gz")

if not selected_map.shape == brain_mask.shape:
    brain_mask = resample_to_img(brain_mask, selected_map, interpolation='nearest')
    print("Brain mask resampled to match input mages.")

# Initialize NiftiMasker with the brain mask, and fit it once
nifti_masker = NiftiMasker(standardize=False,
            mask_img=brain_mask,
            memory_level=0,
            smoothing_fwhm=None)

nifti_masker.fit()
    
n_iterations = 10

wholebrain = 1

def perform_iteration(iteration, output_dir, nifti_masker, split_maps_paths):
    """
    Performs a single iteration of processing, which includes selecting one map per subject/split,
    applying a brain mask, accumulating the masked data, and computing the mean across the selected maps.
    
    Parameters:
    - iteration (int): The current iteration number. Used for output file naming and tracking.
    - output_dir (str): The directory where output files will be saved.
    - brain_mask (numpy.ndarray): A boolean array where True values indicate voxels within the brain.
                                  This mask is applied to each map to focus analysis on brain voxels only.

    Returns:
    - A dictionary containing:
      - The iteration number.
      - The duration it took to complete the iteration.
      - The paths of the selected maps for this iteration.
      - The mean accuracy across the selected maps, considering only brain voxels.
    """

    start_time = time.time()  # Start timing the iteration
    
    # Initialize a 3D array to accumulate masked data across all selected maps
    sum_of_masked_data = np.zeros(example_img.shape)
    mask_voxels_count = np.zeros(example_img.shape)  # To count the non-zero entries for accurate averaging

    selected_maps = []  # Initialize a list to store the paths of selected maps

    # Randomly select one map per subject/split and accumulate
    for split_id, map_paths in split_maps_paths.items():
        selected_map_path = random.choice(map_paths)  # Randomly choose a map for the current split
        selected_maps.append(selected_map_path)  # Store the selected map path for reference
        
        ####INCLUDE MASK TO GET ONLY BRAIN VALUES!!!! ######
        
        masked_data = apply_mask(selected_map_path, nifti_masker.mask_img_)
        mask_indices = np.where(nifti_masker.mask_img_.get_fdata() > 0)

        # Reconstruct the 3D volume for the current map's masked data
        reconstructed_volume = np.zeros(example_img.shape)
        reconstructed_volume[mask_indices] = masked_data

        sum_of_masked_data += reconstructed_volume
        mask_voxels_count[mask_indices] += 1

    # Avoid division by zero
    mask_voxels_count[mask_voxels_count == 0] = 1

    # Compute the mean of accumulated data
    averaged_map_data = sum_of_masked_data / mask_voxels_count
    
    #### NEED TO RECONSTRUCT AVERAGED_MAP_DATA INTO 3D TO SAVE THE NII FILE ####

    # Create a new NIfTI image for the averaged map data and save it
    averaged_map = nib.Nifti1Image(averaged_map_data, example_img.affine)
    averaged_map_filename = f"permuted_mean_group_accuracy_map_{iteration+1:03d}.nii"
    averaged_map_path = os.path.join(output_dir, averaged_map_filename)
    averaged_map.to_filename(averaged_map_path)

    end_time = time.time()  # Mark the end time of the iteration
    duration = end_time - start_time  # Calculate the duration of this iteration

    # Return iteration details, including timing and the mean group accuracy
    return {
        "Iteration": iteration + 1,
        "Duration (s)": duration,
        "Selected Maps": selected_maps,
        "Mean Group Acc": np.mean(averaged_map_data)
    }
    
    #print(f"Saved permuted group accuracy map for iteration {iteration+1}")
    
# Measure the total processing time
total_start_time = time.time()

# Use ProcessPoolExecutor to run iterations in parallel
iteration_timings = []

with ProcessPoolExecutor() as executor:
    # Start all iterations and return futures
    futures = [executor.submit(perform_iteration, iteration, output_dir, nifti_masker, split_maps_paths) for iteration in range(n_iterations)]
    
    # As each future completes, record its duration
    for future in as_completed(futures):
        iteration_timings.append(future.result())

total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"Total processing time: {total_duration:.2f} seconds.")

# Add total processing time to the timings list
iteration_timings.append({"Iteration": "Total", "Duration (s)": total_duration})

# Save timings to a CSV file
timings_file = os.path.join(output_dir, "timings.csv")
with open(timings_file, 'w', newline='') as file:
    # Define fieldnames that match the timing information
    fieldnames = ["Iteration", "Duration (s)"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    # Exclude 'mean_group_acc' and 'Selected Maps' from the data to be written
    for result in iteration_timings:
        if "Iteration" in result and "Duration (s)" in result:
            writer.writerow({key: result[key] for key in fieldnames})

# Save the collected data to a CSV file, corrected for mean selected maps
maps_csv_file_path = os.path.join(output_dir, "null_distribution_group_mean.csv")
with open(maps_csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['Iteration', 'Selected Maps', 'Mean Group Acc']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for result in iteration_timings:
        if "Selected Maps" and "Mean Group Acc" in result:
            # Process each path in selected_maps to extract only the last two parts
            processed_selected_maps = [
                '/'.join(path.split(os.sep)[-2:])  # Adjusted to include only the last two parts of path for context
                for path in result['Selected Maps']
            ]
            # Convert the list of processed paths to a string
            selected_maps_str = '|'.join(processed_selected_maps)
            prepared_data = {
                'Iteration': result['Iteration'],
                'Selected Maps': selected_maps_str,
                'Mean Group Acc': result['Mean Group Acc']
            }
            writer.writerow(prepared_data)

print(f"Timings saved to {timings_file}")
print(f"Null distribution saved to {maps_csv_file_path}")

Brain mask resampled to match input mages.
Total processing time: 3.32 seconds.
Timings saved to /projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut_test/withinsubj_permut_test/permu_mean_group_acc_maps_test/timings.csv
Null distribution saved to /projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut_test/withinsubj_permut_test/permu_mean_group_acc_maps_test/null_distribution_group_mean.csv


In [143]:
sum_of_masked_data

NameError: name 'mask_voxels_count' is not defined

In [ ]:
null_distribution = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut/withinsubj_permut/permu_mean_group_acc_maps_test/null_distribution_group_mean.csv"

null_distribution_df = pd.read_csv(null_distribution)

# Access the "Mean Group Acc" column
null_distribution_df = null_distribution_df['Mean Group Acc']

# Count the number of rows
num_rows = null_distribution_df.count()
print(f"Number of rows: {num_rows}")

# check mean values of null distributions (should be close to 0.5)


# Compute the mean of the "Mean Group Acc" column
column_mean = null_distribution_df.mean()
print(f"Mean of 'Mean Group Acc': {column_mean}")


In [ ]:
null_distribution = "/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/ISPA_SearchLight_TB_permut/withinsubj_permut/permu_mean_group_acc_maps/null_distribution_group_mean.csv"

null_distribution_df = pd.read_csv(null_distribution)


# Generating figures of sinificant brain regions from SnPM


In [ ]:
result_dir = '/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/snpm_batch_cluster_level'
fname = '/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/snpm_batch_cluster_level/lP_FDR+.img'

#brain_nii = nb.load(result_dir + '/lP_FWE+.img')
img = nb.load(fname)
nb.save(img, fname.replace('.img', '.nii'))
display = plotting.plot_glass_brain(None, display_mode='lyrz')
color = 'r'
display.add_contours(img, filled=True, levels=[1], colors=color)
display.title('regions uncovered by ISPA, cluster-level FWE')
#display.savefig(result_dir + '/{}_{}.png'.format(dataset_name, decoding_type))

In [ ]:

cluster_snpm = '/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/snpm_batch_cluster_level/SnPM_filtered.nii'
cluster_snpm = nb.load(cluster_snpm)

vmax = -np.log10(1 / 10000)  # ~= -np.log10(1 / n_perm)


plotting.plot_stat_map(cluster_snpm,
                       cut_coords=(-9, 0, 5, 10),
                      title='SnPM FWE cluster mass', vmax=vmax, display_mode='x', cmap = "Oranges",
                      black_bg=False, draw_cross = False, dim = 1, alpha=0.85)

In [ ]:

cluster_snpm = '/projetos/PRJ1901_AMID/03_PROCS/TCT_FAPERJ_2023/Postnatal_Affect_Dataset/snpm_batch_cluster_level/SnPM_filtered.nii'
cluster_snpm = nb.load(cluster_snpm)

vmax = -np.log10(1 / 10000)  # ~= -np.log10(1 / n_perm)


plotting.plot_stat_map(cluster_snpm,
                       cut_coords=(-20,-15,-9,5),
                     vmax=vmax, display_mode='z', cmap = "Oranges",
                      black_bg=False, draw_cross = False, dim = 1, alpha=0.85)